In [1]:
import argparse, re
from pathlib import Path
import pandas as pd
import numpy as np

In [2]:
OUT_PATH = Path('Exports')
LANG = 'TR'

QUESTION_FILE_PATH = Path(OUT_PATH / f'{LANG}_Questions.csv')
ANSWER_FILE_PATH1 = Path(OUT_PATH / f'{LANG}_Answers1.csv')
ANSWER_FILE_PATH2 = Path(OUT_PATH / f'{LANG}_Answers2.csv')


In [3]:
# 'agreement', 'annotation_id', 'annotator', 'created_at', 'id',
# 'lead_time', 'question', 'sub_id', 'text', 'title', 'updated_at',
# 'url'
question_df = pd.read_csv(QUESTION_FILE_PATH)

# 'Q_Annotator', 'agreement', 'annotation_id', 'annotator', 'answer',
#  'created_at', 'id', 'lead_time', 'question', 'text', 'title',
#  'updated_at', 'url'
answer_df1 = pd.read_csv(ANSWER_FILE_PATH1)
answer_df2 = pd.read_csv(ANSWER_FILE_PATH2)

answer_df = pd.concat([answer_df1, answer_df2], ignore_index=True)
print('Orig: Q: ', len(question_df), 'A: ', len(answer_df))

# drop duplicate questions, answers and append the empty questions / answers
question_df_filtered = pd.concat([question_df.drop_duplicates(subset=['question']), question_df[question_df['question'].isna()]], ignore_index=False)
question_nans = question_df_filtered[question_df_filtered['question'].isna()]
question_nans['answer'] = '' #[''] * len(question_nans)
answer_df_filtered = pd.concat([answer_df.drop_duplicates(subset=['answer']), answer_df[answer_df['answer'].isna()]], ignore_index=False)

print('Q filtered: ', len(question_df_filtered), 'A filtered: ', len(answer_df_filtered))

if len(question_df_filtered) != len(question_df):
    print(f"Q orig contains {len(question_df)-len(question_df_filtered)} more tasks")
    print('Q Orig: ', len(question_df), 'Filtered: ',  len(question_df_filtered))
    q_dups = question_df[question_df.duplicated(subset=['question'], keep=False)]
    
if len(answer_df_filtered) != len(answer_df):
    print(f"A orig contains {len(answer_df)-len(answer_df_filtered)} more tasks")
    print('A orig: ', len(answer_df), 'Filtered: ',  len(answer_df_filtered))


Orig: Q:  964 A:  304
Q filtered:  965 A filtered:  304
Q orig contains -1 more tasks
Q Orig:  964 Filtered:  965


/var/folders/4d/zhxj67g10gnf67g5fvj667kc0000gn/T/ipykernel_51209/2714868614.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  question_nans['answer'] = '' #[''] * len(question_nans)


In [9]:
# get all existing questions / answers from the answer df (1 taks per question)
# last rows contain no answers / questions
print(f'Len df before: {len(answer_df_filtered)}')
answer_df_filtered = answer_df_filtered.drop_duplicates(subset=['question', 'answer'], keep='first')
print(f'Len df after: {len(answer_df_filtered)}')


# answer_df_filtered (Q+A correct), answer_nans (Q correct, A missing), question_nans (Q missing, A nonexistent)
df = pd.concat([answer_df_filtered, question_nans], ignore_index=True)


# save the new df
df.to_csv(OUT_PATH / f'{LANG}_Answers_filtered.csv', index=True)
len(df)


Len df before: 303
Len df after: 303


1003